
<BR>
<CENTER>
<FONT SIZE = 4, COLOR="#2C32C3">
<BR><B>Northeastern University</B>
<BR><B><font size=4, color=black>Husky-3,</BR>
ALY6010 Prob Theory and Intro Stats</B>
 <BR> Motor Vehicle Collision Crash in New York(2017-2023)
<BR> Final Project — Milestone 2 (Hypothesis Testing)
<BR> Aliha Ansari, Leena Doultani, Rohit Kosamkar, Sapna Chavan, Soumya Guliyan, Baudhik Kevadiya
<BR><font size=4, color=dark red> Date: 03.12.2024
</FONT>
</CENTER>

In [ ]:
# Importing libraries
library(dplyr)

In [2]:
# Reading Dataset 
crash_data <- read.csv("motor_vehicle_crashes_clean_data.csv")

### Q1: Is there a significant difference in the proportion of crashes caused by "Driver Inattention/Distraction" between morning rush hour (8-10 AM) and evening rush hour (5-7 PM)?

· Null Hypothesis (H0): There is no difference in the proportion of crashes caused by "Driver Inattention/Distraction" between morning rush hour and evening rush hour.

· Alternative Hypothesis (H1): There is a difference in the proportion of crashes caused by "Driver Inattention/Distraction" between morning rush hour and evening rush hour.

· Test: Chi-square test of independence or proportions test.

In [3]:
# cleaning crash time
crash_data$crash_time <- gsub("^([0-9]{1,2}):([0-9]{2}).*$", "\\1:\\2:00", crash_data$crash_time)

In [4]:
# Standardizing the format of crash_time (assuming it's in HH:MM format)
crash_data$crash_datetime <- as.POSIXlt(paste(crash_data$crash_date, crash_data$crash_time), format = "%Y-%m-%d %H:%M:%S")

In [5]:
# creating new column Time period based on rush hour at morning & evening
crash_data$time_period <- factor(ifelse(format(crash_data$crash_datetime, "%H:%M") >= "08:00" & 
                                          format(crash_data$crash_datetime, "%H:%M") <= "10:00", 
                                        "Morning Rush Hour",
                                        ifelse(format(crash_data$crash_datetime, "%H:%M") >= "17:00" &
                                                 format(crash_data$crash_datetime, "%H:%M") <= "19:00", 
                                               "Evening Rush Hour", 
                                               "Other")), 
                                 levels = c("Morning Rush Hour", "Evening Rush Hour", "Other"))


In [6]:
# creating Contingency table regards to Driver Inattention/Distraction based on time period
cont_table <- table(crash_data$time_period, crash_data$contributing_factor_vehicle_1 == "Driver Inattention/Distraction")

chi_sq_test <- chisq.test(cont_table)

print(chi_sq_test)


	Pearson's Chi-squared test

data:  cont_table
X-squared = 108.92, df = 2, p-value < 2.2e-16



**Test Interpretation:** Since the p-value is much smaller than the chosen significance level (e.g., 0.05), we reject the null hypothesis. Therefore, we conclude that there is a statistically significant difference in the proportions of crashes caused by "Driver Inattention/Distraction" between morning and evening rush hours. In other words, the time period is associated with the occurrence of crashes caused by "Driver Inattention/Distraction".

###  Q2: Is there a significant difference in the average number of crashes between boroughs in New York City?

· Null Hypothesis (H0): There is no difference in the average number of crashes between boroughs in New York City.

· Alternative Hypothesis (H1): There is a difference in the average number of crashes between boroughs in New York City.

· Welch Two Sample t-test
risons

In [7]:
# filtering data based on borough and dropping blanks for analysis and hypothesis tests
total_crashes <- nrow(crash_data)
crashes_borough <- crash_data %>%
  filter( borough != "")
print(total_crashes)

[1] 1112224


In [8]:
# Aggregating crash data by borough
crashes_by_borough <- crashes_borough %>%
  group_by(borough) %>%
  summarise(number_of_crashes = n())

# let's print the aggregated data
print(crashes_by_borough)


# A tibble: 5 x 2
  borough       number_of_crashes
  <chr>                     <int>
1 BRONX                    118867
2 BROOKLYN                 237025
3 MANHATTAN                137441
4 QUEENS                   199013
5 STATEN ISLAND             26991


In [9]:
# Filtering  and Aggregating Daily Crash Counts for Brooklyn
brooklyn_daily_crashes <- crash_data %>%
  filter(borough == "BROOKLYN") %>%
  group_by(crash_date) %>%
  summarise(daily_crashes = n())

In [10]:
# One sample t-test
city_t_test_result <- t.test(brooklyn_daily_crashes$daily_crashes, mu = 100)
print(city_t_test_result)


	One Sample t-test

data:  brooklyn_daily_crashes$daily_crashes
t = -13.844, df = 2616, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 100
95 percent confidence interval:
 89.23578 91.90675
sample estimates:
mean of x 
 90.57126 



**Test Interpretation:** The one-sample t-test conducted on Brooklyn's daily crash counts reveals a significant deviation from the hypothesized mean of 100 crashes per day, with an observed mean of approximately 90.57 crashes. The test's highly significant p-value (less than 2.2e-16) and a confidence interval ranging from 89.24 to 91.91, which does not encompass the hypothesized mean, strongly indicate that the actual average daily crash count is significantly lower than expected. This finding suggests that Brooklyn experiences fewer crashes than anticipated, highlighting the potential effectiveness of traffic safety measures and urban planning strategies in reducing crash incidences in the borough.

In [11]:
# crashes data for Brooklyn and Manhattan
brooklyn_crashes <- filter(crash_data, borough == "BROOKLYN")$collision_id
manhattan_crashes <- filter(crash_data, borough == "MANHATTAN")$collision_id

# Performing two-sample t-test
borough_t_test_result <- t.test(brooklyn_crashes, manhattan_crashes)
print(borough_t_test_result)


	Welch Two Sample t-test

data:  brooklyn_crashes and manhattan_crashes
t = 42.895, df = 285501, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 44943.43 49247.21
sample estimates:
mean of x mean of y 
  4169799   4122704 



**Test Interpretation:** The Welch Two Sample t-test comparing vehicle crashes between Brooklyn and Manhattan reveals a significant difference, with a large t-statistic of 42.895 and a negligible p-value (less than 2.2e-16), indicating the disparity in crash averages between the two boroughs is highly significant. Specifically, Brooklyn has between 44,943 to 49,247 more crashes on average than Manhattan, as demonstrated by the 95% confidence interval. This statistical analysis robustly supports the conclusion that Brooklyn experiences a significantly higher number of vehicle crashes compared to Manhattan.

### Q3. Is there a significant difference between mean number of persons injured between different months 

· Null Hypothesis (H0): There is no significant difference in the mean number of persons injured between different months.

· Alternative Hypothesis (H1): There is a significant difference in the mean number of persons injured between different months.

· Test: Pairwise Two Sample t-test


In [12]:
# Group the data by month and calculate the average number of crashes for each month
avg_crashes_by_month <- crash_data %>%
  group_by(month) %>%
  summarize(avg_crashes = mean(number_of_persons_injured, na.rm = TRUE))  

In [13]:
# One-sample t-test comparing the average number of crashes for each month against the overall average
overall_avg_crashes <- mean(crash_data$number_of_persons_injured, na.rm = TRUE)  
one_sample_t_test <- t.test(avg_crashes_by_month$avg_crashes, mu = overall_avg_crashes)

In [14]:
# Two-sample t-tests (pairwise) comparing the average number of crashes between each pair of months
injury_t_test <- pairwise.t.test(crash_data$number_of_persons_injured, crash_data$month , p.adjust.method = "bonferroni")  
cat("Two-Sample T-test Results:\n")
print(injury_t_test)


Two-Sample T-test Results:

	Pairwise comparisons using t tests with pooled SD 

data:  crash_data$number_of_persons_injured and crash_data$month 

   1       2       3       4       5       6       7       8       9      
2  1.00000 -       -       -       -       -       -       -       -      
3  0.01983 9.8e-05 -       -       -       -       -       -       -      
4  0.00176 0.26278 1.8e-12 -       -       -       -       -       -      
5  < 2e-16 2.4e-11 < 2e-16 0.00197 -       -       -       -       -      
6  < 2e-16 < 2e-16 < 2e-16 1.1e-10 0.18398 -       -       -       -      
7  < 2e-16 < 2e-16 < 2e-16 < 2e-16 9.0e-13 0.00013 -       -       -      
8  < 2e-16 < 2e-16 < 2e-16 < 2e-16 < 2e-16 1.8e-08 1.00000 -       -      
9  < 2e-16 < 2e-16 < 2e-16 < 2e-16 8.9e-14 2.9e-05 1.00000 1.00000 -      
10 < 2e-16 < 2e-16 < 2e-16 9.9e-15 0.00137 1.00000 0.03447 3.2e-05 0.01067
11 < 2e-16 1.7e-11 < 2e-16 0.00132 1.00000 0.35279 7.5e-12 < 2e-16 8.2e-13
12 < 2e-16 7.6e-12 < 2e-16 

**Test Interpretation:**  
Each cell in the table represents the p-value obtained from comparing the average number of persons injured between two months.
The rows and columns represent the months being compared.
For example, the p-value in row "04" and column "05" (5.2e-15) indicates the significance level of the difference in the average number of persons injured between April and May.
The p-values are adjusted using the Bonferroni correction method to account for multiple comparisons.

Many of the p-values are significantly smaller than 0.05, indicating significant differences in the mean number of persons injured between the corresponding months. Therefore, based on these results, we would likely reject the null hypothesis and conclude that there are significant differences in the mean number of persons injured between multiple pairs of months.

### Q4. Whether the mean total injury in the latest year significantly differs from previous year ?

Null Hypothesis (H0): The mean total injury in the latest year is equal to 0.7 

Alternative Hypothesis (H1): The mean total injury in the latest year is not equal to.7

Test: One Sample t-test0.

In [15]:
# Let's find out previous year mean of total injury

df_prev_year <- crash_data %>%
  filter(year %in% c(2018, 2019, 2020))
mean(df_prev_year$total_injury)

[1] 0.60411

In [16]:
df_latest_year <- crash_data %>%
  filter(year %in% c(2021, 2022, 2023))

In [17]:
# Performing one-sample t-test
t_test_result <- t.test(df_latest_year$total_injury, mu = 0.6)

t_test_result


	One Sample t-test

data:  df_latest_year$total_injury
t = 134.16, df = 310992, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0.6
95 percent confidence interval:
 0.9876796 0.9991754
sample estimates:
mean of x 
0.9934275 


**Test Interpretation:** Since the p-value is extremely small (less than any conventional significance level), we reject the null hypothesis. This suggests strong evidence that the true mean total injury in the latest year is not equal to 0.6. The 95% confidence interval also supports this conclusion, as it does not include the value 0.6. Therefore, we conclude that there is a significant difference in the mean total injury compared to the hypothesized value of 0.6.

**References:**  
1. https://www.rdocumentation.org/packages/stats/versions/3.6.2/topics/t.test
2. https://www.datanovia.com/en/lessons/pairwise-t-test
3. https://www.voxco.com/blog/welchs-t-test/
